# A Deep dive into the Atomic Agents framework.
This series aims to provide an in-depth understanding of each building block within the Atomic Agents framework, in this notebook we will dive into Tools and create a simple CalculatorTool. 
Whether you're a seasoned developer or new to AI agent development, this series will offer valuable insights into how to effectively utilize and extend the Atomic Agents framework for your AI applications.

If you just started checkout the BaseAgent notebook first: 

'./A_deep_dive_into_atomic_agents-BaseAgent.ipynb'

# What is the purpose and functionality of Tools.
Tools in multi-agent compound AI systems serve critical functions that enhance specialization, facilitate collaboration, improve efficiency, ensure robustness, and support decision-making. By integrating these tools, multi-agent systems can tackle complex tasks more effectively, making them indispensable in various domains, from enterprise applications to real-world problem-solving scenarios.

Some examples are:
- Data Retrieval and Processing
- Communication and Coordination
- Task Automation and Execution
- Decision Support and Optimization
- Monitoring and Debugging

# BaseTool
Let's have a look at what the `BaseTool` provides, then we will create the Calculator tool.

#### Attributes
- **input_schema** (Type[BaseAgentIO]): The schema for the input data.
- **output_schema** (Type[BaseAgentIO]): The schema for the output data.
- **tool_name** (str): The name of the tool, derived from the input schema's title.
- **tool_description** (str): The description of the tool, derived from the input schema's description or overridden by the user.

#### Public Functions
- **run()**:  Runs the chat agent with the given user input.

Here is an overview of what we have to define to be able to create a tool.
To create the `BaseTool` we need to prepare all the parts we need to configure and instantiate the Agent.

![BaseTool overview schema](../assets/baseTool.png 'BaseTool overview schema')

So let's get starting, first we need to install the packages we need.

In [1]:
# Install the necessary packages
%pip install atomic-agents openai instructor Groq

Note: you may need to restart the kernel to use updated packages.


Next we will do the nessesary imports

In [2]:
# we need to be able to load an API_key from a .env file 
from dotenv import load_dotenv
# we need the os to get to out environment variables
import os
# A library for data validation using Python type annotations. Here, BaseModel and Field are used to define and validate the structure of input and output data.
from pydantic import Field
# A library for rich text and beautiful formatting in the terminal. 
from rich.console import Console
# A library for symbolic mathematics in Python. sympify is used to parse a string into a mathematical expression.
from sympy import sympify
# 
from atomic_agents.agents.base_agent import BaseAgentIO
# we need the BaseTool because it's the base for creating a Tool
# we need the BaseToolConfig to create the configuration for the BaseTool
from atomic_agents.lib.tools.base import BaseTool, BaseToolConfig
# is used to print formatted output.
console = Console()
# we need to load the environment variables from the .env file
load_dotenv()

True

So let's start with the `input_schema` extend it from the BaseAgentIO and define the title and description.

In [3]:
# A class that defines the structure of the input data for the calculator tool.
class CalculatorToolInputSchema(BaseAgentIO):
    # A string field that holds the mathematical expression to evaluate. The Field function provides metadata like a description.
    expression: str = Field(
        ...,
        description = "Mathematical expression to ecvaluate. For example '2 + 2'.",
    )
    #  An inner class that provides additional configuration for the schema, including a title and description.
    class Config:
        title = "CalculatorTool"
        description = (
            "Tool for performing calculations. Supports basic arithmetic operations "
            "like addition, subtraction, multiplication, and division, but also more "
            "complex operations like exponentiation and trigonometric functions. "
            "Use this tool to evaluate mathematical expressions."
        )
        json_schema_extra = {"title": title, "description": description}

Next we need to define the output_schema.

In [4]:
#  A class that defines the structure of the output data for the calculator tool.
class CalculatorToolOutputSchema(BaseAgentIO):
    # A string field that holds the result of the calculation.
    result: str = Field(..., description="Result of the calculation.")

Now we need to define the tool_logic

In [5]:
class CalculatorToolConfig(BaseToolConfig):
    pass

There is no specific logic to define so we can just pass this function.

Now it's time to put it all together.

In [6]:
# A class that implements the calculator tool.
class CalculatorTool(BaseTool):
    # Specifies the expected input schema.
    input_schema = CalculatorToolInputSchema
    # Specifies the expected output schema.
    output_schema = CalculatorToolOutputSchema
    
    # The constructor method that initializes the tool. It optionally takes a configuration parameter.
    def __init__(self, config: CalculatorToolConfig = CalculatorToolConfig()):
        super().__init__(config)
    
    # A method that takes an instance of CalculatorToolInputSchema, parses the mathematical expression using sympify, evaluates it, and returns the result wrapped in CalculatorToolOutputSchema.
    def run(self, params: CalculatorToolInputSchema) -> CalculatorToolOutputSchema:
        parsed_expression = sympify(str(params.expression))
        result = parsed_expression.evalf()
        return CalculatorToolOutputSchema(result=str(result))

The last step is to create some usage example of the tool.

In [7]:
# This block ensures that the code inside it runs only when the script is executed directly, not when imported as a module.
if __name__ == "__main__":
    rich_console = Console()
    # Creates an instance of the CalculatorTool.
    tool = CalculatorTool()
    # Creates an instance of the input schema with the expression "2 + 2".
    input_data = CalculatorToolInputSchema(expression="2 + 2")
    # Runs the calculator tool with the input data and stores the result.
    output_data = tool.run(input_data)
    rich_console.print(output_data)

{
  "result": "4.00000000000000"
}

# Tool Summary
This code defines a simple calculator tool using Python classes and several libraries. It validates input and output data structures using Pydantic, parses and evaluates mathematical expressions using SymPy, and prints the results using Rich. The main execution block demonstrates how to use the tool by creating an instance, running it with an example input, and printing the result.